# LSTM PyTorch

In [20]:
import numpy as np
import datetime
now = datetime.datetime.now()

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import torch.optim as optim
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# GPU check

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用デバイス:", device)

使用デバイス: cuda


# Dataset

In [4]:
outname ="lstm"

outdir = 'C:/Users/zankyo/Desktop/yamamoto/model/' +outname+ now.strftime('%Y%m%d_%H%M%S')
print(outdir)

C:/Users/zankyo/Desktop/yamamoto/model/lstm20201116_170017


In [5]:
audio_len = 2**15
usedata_num = 15000
sample_rate = 16000

In [21]:
datasets_dir = 'C:/Users/zankyo/Desktop/yamamoto/datasets/datasets129x257.npz'
datasets = np.load(datasets_dir)

data, label = datasets['data'], datasets['label']

data_list = []
for i in tqdm(range(len(data))):
    data_list.append(data[i])
    
label_list = []
for i in tqdm(range(len(data))):
    label_list.append(label[i])
    
tensor_data = torch.stack([torch.Tensor(i) for i in data_list])
tensor_label = torch.stack([torch.Tensor(i) for i in label_list])

mydataset = utils.TensorDataset(tensor_data,tensor_label)

train_dataset, val_dataset,test_dataset = utils.random_split(mydataset,[12000,2500,500])

C:\Users\zankyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  ..\aten\src\ATen\native\Copy.cpp:162.)
  if __name__ == '__main__':


# Dataloader

In [22]:
batch_size = 50

train_dataloader = utils.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader = utils.DataLoader(val_dataset,batch_size=batch_size,shuffle=True)
test_dataloader = utils.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

#辞書型変数にまとめる
dataloaders_dict = {"train":train_dataloader,
                    "val":val_dataloader,
                    "test":test_dataloader}

# 動作の確認

Batch, Freq(Feature), Time(Sequence)

In [23]:
batch_iterator = iter(dataloaders_dict["train"])
datas,labels=next(batch_iterator)
print(datas.size())
print(labels.size())

torch.Size([50, 129, 257])
torch.Size([50, 129, 257])


# Net Work Model

In [48]:
feat_size = x_train.shape[1]
sequence_len = x_train.shape[2]

class Net(nn.Module):
    def __init__(self):
        
        super(Net,self).__init__()
        self.seq_len = sequence_len
        self.feature_size = feat_size
        self.output_size = feat_size
        self.hidden_layer_size = 150 #隠れ層のサイズ
        self.lstm_layers = 3 #LSTMのレイヤー数
        
        self.lstm = nn.LSTM(self.feature_size,
                           self.hidden_layer_size,
                           num_layers = self.lstm_layers)
        
        self.fc = nn.Linear(self.hidden_layer_size,self.output_size)
        self.sigmoid = nn.Sigmoid()
        
    def init_hidden_cell(self,batch_size): #LSTMの隠れ層hidden、記憶セルcellを初期化
        hidden = torch.zeros(self.lstm_layers,batch_size,self,hidden_layer_size)
        cell = torch.zeros(self.lstm_layers,batch_size,self,hidden_layer_size)
        return hidden, cell
    
    def forword(self,x):
        batch_size = x.shape[0]
        self.hidden_cell = self.init_hidden_cell(batch_size)
        # 入力は(Batch, Seqence, Feature)
#         x = x.view(batch_size, self.seq_len,self.feature_size)
        x = x.permute(0,2,1) # 次元の入れ替え
        # Batch, Freq(Feature), Time(Sequence)になってるので
        # 入力は(Batch, Seqence, Feature)
        
        lstm_out, (h_n, c_n) = self.lstm(x,self.hidden_cell)
        
        x = h_n[-1,:,:]
        
        x = self.fc(x)
        
        x=self.sigmoid(x)
        
        return x

In [49]:
net = Net().to(device)
net.double()

Net(
  (lstm): LSTM(129, 150, num_layers=3)
  (fc): Linear(in_features=150, out_features=129, bias=True)
  (sigmoid): Sigmoid()
)

# 損失関数の定義

In [50]:
criterion = nn.L1Loss #平均絶対値誤差

# 最適化手法の設定

In [51]:
optimaizer = optim.Adam(net.parameters(),lr = 0.001)

# 学習・検証の実施

In [52]:
def train_model(net, dataloaders_dict, criterion, device, num_epochs):
    
    #epochループ
    for epoch in range(num_epochs):
        print('Epoch{}/{}'.format(epoch+1, num_epochs))
        print('-------------')
        
        #epochごとの学習と検証ループ
        for phase in ['train','val']:
            if phase == 'train':
                net.train() #モデルをトレーニングモードに
            else:
                net.eval() #モデルを検証モードに
                
            epoch = 0.0 #epochの損失和
            epoch_corrects = 0 #epochの正解数
            
            # 未学習時の検証性能を確かめるためepoch=0での訓練はスキップ
            if (epoch == 0) and (phase=='train'):
                continue
                
            # データローダーからミニバッチを取り出す
            for i,(inputs, labels) in tqdm(enumerate(dataloaders_dict[phase])):
                
                #GPUにキャスト
                inputs = inputs.to(device)
                labels = inputs.to(device)
                
                #optimizerを初期化
                optimaizer.zero_grad()
                
                # 順伝搬計算(forword)
                with torch.set_grad_enabled(phase == 'train'): #訓練モードのみ勾配計算
                    outputs = net(inputs)
                    loss = criterion(outputs, labels) #損失の計算
                    _, preds = torch.max(outputs,1) #ラベルの予測
                    
                #訓練時は逆伝搬(バックプロパゲーション)
                if phasa == 'train':
                    loss.backward()
                    optimaizer.step()
                    
                # Iteration結果の計算
                # lossの計算を更新
                epoch_loss += loss.item() * input.size(0)
                # 正解数の合計を更新
                epoch_corrects += touch.sum(preds == labels.data)
            
            # epochごとのlossと正答率を表示
            epoch_loss = epoch_loss/len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double()/len(dataloaders_dict[phase].dataset)
            
            print('{}Loss: {:.4f} Acc:{:,.4f}'.format(phase, epoch_loss,epoch_acc))
                    

# 学習・検証の実行

In [53]:
num_epochs = 3
train_model(net, dataloaders_dict, criterion, device, num_epochs = num_epochs)

Epoch1/3
-------------


NotImplementedError: 